In [1]:
import pandas as pd

In [2]:
from datasets import load_dataset

dataset = load_dataset("raghavtangri/For_fine_tune")
df = pd.DataFrame(dataset['train'])


In [3]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
from transformers import T5ForConditionalGeneration


In [5]:
def preprocess_function(examples):
    inputs = examples["Sentences"]
    targets = examples["Explanation"]
    
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Tokenize labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [6]:
# Print samples from the dataset to check for non-integer values
print(tokenized_dataset['train'][0])
print(tokenized_dataset['test'][0])


{'Sentences': "[i for i, j in enumerate(['foo', 'bar', 'baz']) if j == 'bar']\n", 'Explanation': "This snippet uses a list comprehension to find the indices of elements in the list ['foo', 'bar', 'baz'] that match the value 'bar'. The list comprehension iterates over the list with enumerate(), which provides both the index (i) and the element (j). It includes an if condition to filter for elements where j equals 'bar'. The result is a list of indices where the element is 'bar'. In this case, 'bar' is located at index 1, so the output is [1], indicating the position of 'bar' in the list.\n\n\n\n\n\n\n\n", 'input_ids': [784, 23, 21, 3, 23, 6, 3, 354, 16, 3, 35, 440, 49, 342, 599, 6306, 31, 89, 32, 32, 31, 6, 3, 31, 1047, 31, 6, 3, 31, 115, 9, 172, 31, 908, 61, 3, 99, 3, 354, 3274, 2423, 3, 31, 1047, 31, 908, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [100, 3

In [7]:
from datasets import DatasetDict

def remove_columns(example):
    return {col: example[col] for col in ['input_ids', 'attention_mask', 'labels']}

# Map the function to remove columns
cleaned_tokenized_dataset = tokenized_dataset.map(lambda x: remove_columns(x), batched=True, remove_columns=['Sentences', 'Explanation'])

In [8]:
cleaned_tokenized_dataset.keys()

dict_keys(['train', 'test'])

In [9]:
from transformers import T5ForConditionalGeneration

In [10]:
from transformers import TFT5ForConditionalGeneration

model = TFT5ForConditionalGeneration.from_pretrained("t5-small")


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [21]:
from transformers import TrainingArguments

ImportError: cannot import name 'TfTrainingArguments' from 'transformers' (c:\Users\Raghav Tangri\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\__init__.py)

In [12]:

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    #predict_with_generate=True,
)

c:\Users\Raghav Tangri\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
import evaluate

In [14]:
metric = evaluate.load("accuracy")

In [15]:
def compute_metrics(eval_pred):
  # label_ids, metrics = eval_pred
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [22]:
from transformers import Trainer

In [23]:
trainer = Trainer(
    model=TFT5ForConditionalGeneration.from_pretrained("google-t5/t5-small"),
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics,
)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


ImportError: 
Trainer requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFTrainer".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


In [ ]:
trainer.train()